In [30]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from langchain_google_vertexai import ChatVertexAI
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver

In [31]:
load_dotenv()

llm=ChatVertexAI(model_name="gemini-2.5-pro")

In [32]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explaination:str

In [33]:
def generate_joke(state:JokeState):
    prompt=f"generate a joke on the topic {state['topic']}"
    response=llm.invoke(prompt).content
    return {'joke':response}

In [34]:
def generate_explaination(state:JokeState):
    prompt=f"Write an explaination for the joke - {state["joke"]}"
    response=llm.invoke(prompt).content
    return {"explaination":response}

In [35]:
graph=StateGraph(JokeState)

graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explaination',generate_explaination)

graph.add_edge(START,'generate_joke')
graph.add_edge('generate_joke','generate_explaination')
graph.add_edge('generate_explaination',END)

checkpointer=InMemorySaver()
workflow=graph.compile(checkpointer=checkpointer)

In [36]:
config1={"configurable":{'thread_id':'2'}}
workflow.invoke({'topic':"Mango"},config=config1)

{'topic': 'Mango',
 'joke': 'Why did the peach break up with the mango?\n\nBecause he could never commit, he was always ready for a **man-to-go!**',
 'explaination': 'Of course! Here is an explanation of the joke.\n\n### The Breakdown of the Joke\n\nThis joke is a classic **pun**, which means it relies on a play on words. The humor comes from the double meaning of the punchline.\n\nHere\'s how it works:\n\n1.  **The Setup:** The joke creates a personified scenario where two fruits, a peach and a mango, are in a romantic relationship. The peach breaks up with the mango, and we\'re asked why. This sets up the expectation of a common reason for a breakup.\n\n2.  **The Punchline:** "Because he could never commit, he was always ready for a **man-to-go!**"\n\n3.  **The Pun Explained:**\n    *   The key is that the phrase **"man-to-go"** sounds almost identical to the word **"mango."**\n    *   The phrase also has a literal meaning that fits the context of the joke. "A man to go" perfectly de

In [37]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Mango', 'joke': 'Why did the peach break up with the mango?\n\nBecause he could never commit, he was always ready for a **man-to-go!**', 'explaination': 'Of course! Here is an explanation of the joke.\n\n### The Breakdown of the Joke\n\nThis joke is a classic **pun**, which means it relies on a play on words. The humor comes from the double meaning of the punchline.\n\nHere\'s how it works:\n\n1.  **The Setup:** The joke creates a personified scenario where two fruits, a peach and a mango, are in a romantic relationship. The peach breaks up with the mango, and we\'re asked why. This sets up the expectation of a common reason for a breakup.\n\n2.  **The Punchline:** "Because he could never commit, he was always ready for a **man-to-go!**"\n\n3.  **The Pun Explained:**\n    *   The key is that the phrase **"man-to-go"** sounds almost identical to the word **"mango."**\n    *   The phrase also has a literal meaning that fits the context of the joke. "A man 

In [38]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Mango', 'joke': 'Why did the peach break up with the mango?\n\nBecause he could never commit, he was always ready for a **man-to-go!**', 'explaination': 'Of course! Here is an explanation of the joke.\n\n### The Breakdown of the Joke\n\nThis joke is a classic **pun**, which means it relies on a play on words. The humor comes from the double meaning of the punchline.\n\nHere\'s how it works:\n\n1.  **The Setup:** The joke creates a personified scenario where two fruits, a peach and a mango, are in a romantic relationship. The peach breaks up with the mango, and we\'re asked why. This sets up the expectation of a common reason for a breakup.\n\n2.  **The Punchline:** "Because he could never commit, he was always ready for a **man-to-go!**"\n\n3.  **The Pun Explained:**\n    *   The key is that the phrase **"man-to-go"** sounds almost identical to the word **"mango."**\n    *   The phrase also has a literal meaning that fits the context of the joke. "A man

In [ ]:
config2={"configurable":{"thread_id":"2"}}
workflow.invoke({'topic':"Potato"},config=config2)

In [ ]:
config1={"configurable":{'thread_id':'2'}}
workflow.invoke({'topic':"Mango"},config=config2)

In [ ]:
workflow.get_state(config1)

### Time Travel

In [40]:
workflow.get_state({"configurable":{"thread_id":"2","checkpoint_id":"1f076dee-62d3-6023-8000-2a681d9d3604"}})

StateSnapshot(values={'topic': 'Mango'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f076dee-62d3-6023-8000-2a681d9d3604'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-11T18:13:36.502369+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f076dee-62d1-6676-bfff-07b04caf166a'}}, tasks=(PregelTask(id='56b5e178-eaf2-92f9-2429-7733d71c66bf', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the peach break up with the mango?\n\nBecause he could never commit, he was always ready for a **man-to-go!**'}),), interrupts=())

In [41]:
workflow.invoke(None,{"configurable":{"thread_id":"2","checkpoint_id":"1f076dee-62d3-6023-8000-2a681d9d3604"}})

{'topic': 'Mango',
 'joke': 'Why did the other fruits get so worried at the party?\n\nBecause they kept asking, "Where did the man-go?"',
 'explaination': 'Of course! Here is an explanation of the joke.\n\n### The Breakdown of the Joke\n\nThis joke is a classic **pun**, which means it relies on a word or phrase that has two different meanings or sounds like another word.\n\nThe key to this joke is the punchline:\n\n**"Where did the man-go?"**\n\nHere\'s how it works:\n\n1.  **The Literal Meaning:** When you hear the phrase "Where did the man go?", you naturally think of a question about a missing person. If a man suddenly vanished from a party, people would understandably get worried. This aligns with the setup of the joke ("Why did the other fruits get so worried?").\n\n2.  **The Punny Meaning:** Since the characters in the joke are fruits, the phrase is meant to be heard as a single word: **"Mango"**. The fruits at the party are simply looking for their friend, the Mango, and are ask

In [43]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Mango', 'joke': 'Why did the other fruits get so worried at the party?\n\nBecause they kept asking, "Where did the man-go?"', 'explaination': 'Of course! Here is an explanation of the joke.\n\n### The Breakdown of the Joke\n\nThis joke is a classic **pun**, which means it relies on a word or phrase that has two different meanings or sounds like another word.\n\nThe key to this joke is the punchline:\n\n**"Where did the man-go?"**\n\nHere\'s how it works:\n\n1.  **The Literal Meaning:** When you hear the phrase "Where did the man go?", you naturally think of a question about a missing person. If a man suddenly vanished from a party, people would understandably get worried. This aligns with the setup of the joke ("Why did the other fruits get so worried?").\n\n2.  **The Punny Meaning:** Since the characters in the joke are fruits, the phrase is meant to be heard as a single word: **"Mango"**. The fruits at the party are simply looking for their friend, th

## Updating State